In [14]:
#  Code with additional functionality
import os
import fitz  # PyMuPDF
import tkinter as tk
from tkinter import simpledialog, messagebox
import webbrowser
import shutil
import re

class PDFViewer:
    def __init__(self, master):
        self.master = master
        self.master.title("PDF Viewer")

        self.open_pdf_button = tk.Button(master, text="Open Current PDF in Browser", command=self.open_pdf_in_browser)
        self.open_pdf_button.pack(pady=10)

        self.extract_button = tk.Button(master, text="Extract Technical Specifications", command=self.extract_technical_specs)
        self.extract_button.pack(pady=10)

        self.move_pdf_button = tk.Button(master, text="Move PDF without Extraction", command=self.move_pdf_without_extraction)
        self.move_pdf_button.pack(pady=10)

        self.current_pdf_label = tk.Label(master, text="")
        self.current_pdf_label.pack(pady=10)

        self.pdf_files = []
        self.load_pdf_files()
        self.update_current_pdf_label()

    def load_pdf_files(self):
        initial_dir = "Tender documents"
        self.pdf_files = [f for f in os.listdir(initial_dir) if f.endswith('.pdf')]
        if not self.pdf_files:
            messagebox.showerror("Error", "No PDF files found in the 'Tender documents' directory.")
            self.master.quit()
            return

    def update_current_pdf_label(self):
        if self.pdf_files:
            self.current_pdf_label.config(text=f"Current PDF: {self.pdf_files[0]}")
            self.extract_button.config(state="normal")
            self.open_pdf_button.config(state="normal")
            self.move_pdf_button.config(state="normal")
        else:
            self.current_pdf_label.config(text="No more PDFs to process")
            self.extract_button.config(state="disabled")
            self.open_pdf_button.config(state="disabled")
            self.move_pdf_button.config(state="disabled")

    def open_pdf_in_browser(self):
        if self.pdf_files:
            pdf_path = os.path.join("Tender documents", self.pdf_files[0])
            webbrowser.open_new(pdf_path)

    def extract_technical_specs(self):
        if self.pdf_files:
            try:
                pdf_path = os.path.join("Tender documents", self.pdf_files[0])
                with fitz.open(pdf_path) as pdf_document:
                    total_pages = len(pdf_document)

                    page_range = simpledialog.askstring("Input", f"Enter page range for technical specifications (e.g., 10-13) (1-{total_pages}):")
                    if page_range is None:
                        return

                    start_page, end_page = self.parse_page_range(page_range, total_pages)
                    if start_page is None or end_page is None:
                        return

                    self.save_technical_specs(pdf_document, start_page, end_page)

                # Move the PDF to the processed folder after extraction
                self.move_processed_pdf(pdf_path, "pdf processed")

                messagebox.showinfo("Success", "Technical specifications extracted and PDF moved successfully.")
                
                # Remove the processed PDF from the list and update the display
                self.pdf_files.pop(0)
                self.update_current_pdf_label()

            except Exception as e:
                messagebox.showerror("Error", f"An error occurred: {str(e)}")

    def parse_page_range(self, page_range, total_pages):
        pattern = r'^(\d+)-(\d+)$'
        match = re.match(pattern, page_range)
        if not match:
            messagebox.showerror("Error", "Invalid page range format. Please use format like '10-13'.")
            return None, None

        start_page = int(match.group(1))
        end_page = int(match.group(2))

        if start_page < 1 or end_page > total_pages or start_page > end_page:
            messagebox.showerror("Error", f"Invalid page range. Please enter values between 1 and {total_pages}.")
            return None, None

        return start_page, end_page

    def save_technical_specs(self, pdf_document, start_page, end_page):
        technical_dir = "technical doc"
        os.makedirs(technical_dir, exist_ok=True)

        output_text = ""
        for page_num in range(start_page, end_page + 1):
            page = pdf_document[page_num - 1]  # Adjust for 0-based indexing
            output_text += f"--- Page {page_num} ---\n"
            output_text += page.get_text()
            output_text += "\n\n"

        output_filename = f"{self.pdf_files[0].replace('.pdf', '_tech_specs.txt')}"
        output_path = os.path.join(technical_dir, output_filename)

        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(output_text)

    def move_processed_pdf(self, pdf_path, destination_folder):
        """
        Moves the PDF to the specified destination folder.
        """
        os.makedirs(destination_folder, exist_ok=True)
        destination_path = os.path.join(destination_folder, os.path.basename(pdf_path))
        shutil.move(pdf_path, destination_path)

    def move_pdf_without_extraction(self):
        """
        Moves the PDF without extraction to the 'not_needed' folder inside 'pdf processed'.
        """
        if self.pdf_files:
            try:
                pdf_path = os.path.join("Tender documents", self.pdf_files[0])
                not_needed_folder = os.path.join("pdf processed", "not_needed")
                
                # Move the PDF to the 'not_needed' folder
                self.move_processed_pdf(pdf_path, not_needed_folder)
                
                messagebox.showinfo("Success", "PDF moved successfully without extraction.")
                
                # Remove the processed PDF from the list and update the display
                self.pdf_files.pop(0)
                self.update_current_pdf_label()

            except Exception as e:
                messagebox.showerror("Error", f"An error occurred: {str(e)}")

def main():
    root = tk.Tk()
    viewer = PDFViewer(root)
    root.mainloop()

if __name__ == "__main__":
    main()


KeyboardInterrupt: 